# EDA and Modelling - Notebook

### Table of Contents

* [Import Libraries and Data](#chapterLibraryData)

* [Chapter 1. Data Context ](#chapter1)
* [Chapter 2. Data Description ](#chapter2)
  * [Section 2.1 Data Loading](#section_2_1)
  * [Section 2.2 Analysis 1 - Volumetric Analysis](#section_2_2)
  * [Section 2.3 Analysis 2 - Univariate Analysis - NonGraphical / Graphical + Data Cleaning](#section_2_3)
   * [Section 2.4 Analysis 3 - Bivariate Analysis - NonGraphical / Graphical](#section_2_4)
   * [Section 2.5 Analysis 4 - Feature Selection](#section_2_5)
   * [Section 2.6 Analysis 5 - Feature Encoding](#section_2_6)
   * [Section 2.7 Analysis 6 - Modelling](#section_2_7)
   * [Section 2.8 Analysis 7 - Fairness-Analysis](#section_2_8)

### Import Libraries and Data <a class="anchor" id="chapterLibraryData"></a>

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
from IPython.display import display
from dython.nominal import associations
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_score, recall_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import warnings
import logging
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from scipy.stats import shapiro
import joblib
import plotly.offline as pyo
warnings.filterwarnings("ignore")
from fairlearn.metrics import (MetricFrame,demographic_parity_difference, equalized_odds_difference)

# I. Data Context

The datasource of this case study is public and sourced from Home Credit (2018), a non-banking international financial institution providing lending services to individuals with little or no credit history. 

The datasource can be found on Kaggle (https://www.kaggle.com/competitions/home-credit-default-risk/overview), where it was featured in a past competition addressing the task of predicting applicants' repayment capability for a requested loan.

The datasource consists of several tables, of which 2 primary ones were selected: application_train and bureau. 

1. application_train.csv

- consists of 307.511 rows and 122 columns, which includes customer details - past loan characteristics and personal facts at the time of the application
- "TARGET", serves as the target of the case study's prediction task, having two distinct values: 1 - the customer had repayment difficulties and 0 - the customer did not have repayment difficulties
- "SK_ID_CURR" is a unique column used to identify past loans issued by Home Credit

2. bureau.csv

- consists of 1.716.428 rows and 17 columns, which include details from loans of Home Credit's customers granted by other financial institutions
- "SK_ID_BUREAU", unique column used to identify the table's specific loans
- "SK_ID_CURR" refers to the past loans issued by Home Credit

<img src="../figure/erd_picture.png" alt="ERD" width="340" height="150">

# II. Data Description

### Data Loading

In [ ]:
application = pd.read_csv("../data/raw/application_train.csv")
bureau = pd.read_csv("../data/raw/bureau.csv")
print("Shape application: " + str(application.shape))
print("Shape bureau: " + str(bureau.shape))
application['SK_ID_CURR'] = application['SK_ID_CURR'].astype('object')
bureau['SK_ID_CURR'] = bureau['SK_ID_CURR'].astype('object') 
bureau['SK_ID_BUREAU'] = bureau['SK_ID_BUREAU'].astype('object') 
display(application.head())
display(bureau.head())

In [ ]:
def categorical_agg(x): 
    return x.mode().iloc[0] if (not x.mode().empty) and (not x.isnull().all()) else np.nan
def numerical_agg(x): 
    return x.mean() if not x.isnull().all() else np.nan

results = {}
for column in bureau.select_dtypes(exclude=['int', 'float']).columns: 
    results[column] = bureau.groupby('SK_ID_CURR')[column].apply(categorical_agg) 
for column in bureau.select_dtypes(include=['int', 'float']).columns: 
    results[column] = bureau.groupby('SK_ID_CURR')[column].apply(numerical_agg) 
bureau = pd.DataFrame(results)
bureau.reset_index(drop=True, inplace=True) 

data = application.merge(bureau, on='SK_ID_CURR', how='inner')

flag_columns = [f'FLAG_DOCUMENT_{i}' for i in range(2, 22)] 
for col in flag_columns: 
    data[col] = data[col].astype('object') 
columns_to_convert = [ 
    'SK_ID_CURR', 'SK_ID_BUREAU', 'FLAG_MOBIL', 
    'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
    'FLAG_PHONE', 'FLAG_EMAIL', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION',
    'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY'
]
for col in columns_to_convert: 
    data[col] = data[col].astype('object')

data.drop(columns=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'], inplace=True) 
flag_columns = [col for col in data.columns if col.startswith('FLAG_DOCUMENT_')]
data['FD_TOTAL'] = data[flag_columns].sum(axis=1)
columns_to_drop = [col for col in data.columns if col.startswith('FLAG_DOCUMENT_')]
data.drop(columns=columns_to_drop, inplace=True) 

data.rename(columns={col: col+'_app' for col in application.columns if col != 'SK_ID_CURR'}, inplace=True) 
data.rename(columns={col: col+'_bur' for col in bureau.columns if col != 'SK_ID_CURR'}, inplace=True) 

print("Shape data: " + str(data.shape))

data.head() 

### Analysis 1 - Volumetric Analysis: 

This section examines the “surface” properties of the acquired data.

In [ ]:
def volumetric_analysis(df):
    print("Data Types:\n" + str(df.dtypes))
    print("\nRows and Columns: \n" + str(df.shape))
    print("\nColumns Names:\n" + str(df.columns))
    
    null_percentages = df.isnull().mean().sort_values(ascending=False)
    print("\nNull Values Percentage (Descending Order):\n" + str(null_percentages))
    null_values_desc = df.isnull().sum().sort_values(ascending=False)
    print("\nNull Values (Descending Order):\n" + str(null_values_desc))
    print("\nDuplicate Values:\n" + str(df[df.duplicated()].shape))

volumetric_analysis(data)

### Analysis 2 - Univariate Analysis - NonGraphical / Graphical + Data Cleaning

This section analyses several columns of the dataset separately using descriptive statistics based on data type and visual preference: count of unique values, frequency distributions (ungrouped - using bar charts and grouped - using histograms), measures of center (mode, meadian, mode) and measures of dispersion (standard deviation).

In [ ]:
data['TARGET_app'] = data['TARGET_app'].replace({0: 'non-default', 1: 'default'})

#### 2.1 Qualitative data - NonGraphical

In [ ]:
data_unique = []
object_column_count = 0
for col in data.select_dtypes(include='object').columns:
    object_column_count += 1
    data_unique.append([col, data[col].nunique()])

df_cat_unique_values = pd.DataFrame(data_unique, columns=['Variable_name', 'Count of Unique values'])
print("Number of object columns:", object_column_count)
print(df_cat_unique_values)

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["TARGET_app"].value_counts(),'Relative Frequency': data["TARGET_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'TARGET_app' 
df 

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["NAME_CONTRACT_TYPE_app"].value_counts(),'Relative Frequency': data["NAME_CONTRACT_TYPE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'NAME_CONTRACT_TYPE_app' 
df

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["CODE_GENDER_app"].value_counts(),'Relative Frequency': data["CODE_GENDER_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'CODE_GENDER_app' 
df

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_OWN_CAR_app"].value_counts(),'Relative Frequency': data["FLAG_OWN_CAR_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_OWN_CAR_app' 
df 

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_OWN_REALTY_app"].value_counts(),'Relative Frequency': data["FLAG_OWN_REALTY_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_OWN_REALTY_app' 
df

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["NAME_TYPE_SUITE_app"].value_counts(),'Relative Frequency': data["NAME_TYPE_SUITE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'NAME_TYPE_SUITE_app' 
df 

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["NAME_INCOME_TYPE_app"].value_counts(),'Relative Frequency': data["NAME_INCOME_TYPE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'NAME_INCOME_TYPE_app' 
df.head() 

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["NAME_EDUCATION_TYPE_app"].value_counts(),'Relative Frequency': data["NAME_EDUCATION_TYPE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'NAME_EDUCATION_TYPE_app' 
df.head() 

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["NAME_FAMILY_STATUS_app"].value_counts(),'Relative Frequency': data["NAME_FAMILY_STATUS_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'NAME_FAMILY_STATUS_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["NAME_HOUSING_TYPE_app"].value_counts(),'Relative Frequency': data["NAME_HOUSING_TYPE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'NAME_HOUSING_TYPE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_MOBIL_app"].value_counts(),'Relative Frequency': data["FLAG_MOBIL_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_MOBIL_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_EMP_PHONE_app"].value_counts(),'Relative Frequency': data["FLAG_EMP_PHONE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_EMP_PHONE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_WORK_PHONE_app"].value_counts(),'Relative Frequency': data["FLAG_WORK_PHONE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_WORK_PHONE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_CONT_MOBILE_app"].value_counts(),'Relative Frequency': data["FLAG_CONT_MOBILE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_CONT_MOBILE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_PHONE_app"].value_counts(),'Relative Frequency': data["FLAG_PHONE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_PHONE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FLAG_EMAIL_app"].value_counts(),'Relative Frequency': data["FLAG_EMAIL_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FLAG_EMAIL_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["OCCUPATION_TYPE_app"].value_counts(),'Relative Frequency': data["OCCUPATION_TYPE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'OCCUPATION_TYPE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["REGION_RATING_CLIENT_app"].value_counts(),'Relative Frequency': data["REGION_RATING_CLIENT_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'REGION_RATING_CLIENT_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["REGION_RATING_CLIENT_W_CITY_app"].value_counts(),'Relative Frequency': data["REGION_RATING_CLIENT_W_CITY_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'REGION_RATING_CLIENT_W_CITY_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["WEEKDAY_APPR_PROCESS_START_app"].value_counts(),'Relative Frequency': data["WEEKDAY_APPR_PROCESS_START_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'WEEKDAY_APPR_PROCESS_START_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["REG_REGION_NOT_LIVE_REGION_app"].value_counts(),'Relative Frequency': data["REG_REGION_NOT_LIVE_REGION_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'REG_REGION_NOT_LIVE_REGION_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["REG_REGION_NOT_WORK_REGION_app"].value_counts(),'Relative Frequency': data["REG_REGION_NOT_WORK_REGION_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'REG_REGION_NOT_WORK_REGION_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["LIVE_REGION_NOT_WORK_REGION_app"].value_counts(),'Relative Frequency': data["LIVE_REGION_NOT_WORK_REGION_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'LIVE_REGION_NOT_WORK_REGION_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["REG_CITY_NOT_LIVE_CITY_app"].value_counts(),'Relative Frequency': data["REG_CITY_NOT_LIVE_CITY_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'REG_CITY_NOT_LIVE_CITY_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["REG_CITY_NOT_WORK_CITY_app"].value_counts(),'Relative Frequency': data["REG_CITY_NOT_WORK_CITY_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'REG_CITY_NOT_WORK_CITY_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["LIVE_CITY_NOT_WORK_CITY_app"].value_counts(),'Relative Frequency': data["LIVE_CITY_NOT_WORK_CITY_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'LIVE_CITY_NOT_WORK_CITY_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["ORGANIZATION_TYPE_app"].value_counts(),'Relative Frequency': data["ORGANIZATION_TYPE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'ORGANIZATION_TYPE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["FONDKAPREMONT_MODE_app"].value_counts(),'Relative Frequency': data["FONDKAPREMONT_MODE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'FONDKAPREMONT_MODE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["HOUSETYPE_MODE_app"].value_counts(),'Relative Frequency': data["HOUSETYPE_MODE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'HOUSETYPE_MODE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["WALLSMATERIAL_MODE_app"].value_counts(),'Relative Frequency': data["WALLSMATERIAL_MODE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'WALLSMATERIAL_MODE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["EMERGENCYSTATE_MODE_app"].value_counts(),'Relative Frequency': data["EMERGENCYSTATE_MODE_app"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'EMERGENCYSTATE_MODE_app' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["CREDIT_ACTIVE_bur"].value_counts(),'Relative Frequency': data["CREDIT_ACTIVE_bur"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'CREDIT_ACTIVE_bur' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["CREDIT_CURRENCY_bur"].value_counts(),'Relative Frequency': data["CREDIT_CURRENCY_bur"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'CREDIT_CURRENCY_bur' 
df.head()

In [ ]:
df = pd.DataFrame({'Frequency Distribution': data["CREDIT_TYPE_bur"].value_counts(),'Relative Frequency': data["CREDIT_TYPE_bur"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})
df.index.name = 'CREDIT_TYPE_bur' 
df.head()

#### 2.2 Qualitative data - Graphical

In [ ]:
value_counts = data["TARGET_app"].value_counts(normalize=True)
set3_palette = px.colors.qualitative.Set3
colors = [set3_palette[4], set3_palette[5]]  

fig = px.bar(x=value_counts.index, 
             y=value_counts * 100, 
             text=(value_counts * 100).round(2).astype(str) + '%', 
             color=value_counts.index,
             color_discrete_sequence=colors,
             height=750, 
             width=600, 
             title='% Distribution of TARGET')

fig.update_traces(textfont_size=12, textposition="inside")
fig.update_layout(yaxis_title='Percentage', xaxis_title='TARGET',xaxis_title_standoff=377)
fig.update_xaxes(tickvals=['No', 'Yes'], ticktext=['No', 'Yes'])

fig.show()

#### 2.3 Qualitative data - Data Cleaning

In [ ]:
data = data[data['CODE_GENDER_app'] != 'XNA']

In [ ]:
def feature_type_split(data):
    cat_list = []
    dis_num_list = []
    num_list = []
    for i in data.columns.tolist():
        if data[i].dtype == 'object':
            cat_list.append(i)
        elif data[i].nunique() < 25:
            dis_num_list.append(i)
        else:
            num_list.append(i)
    return cat_list, dis_num_list, num_list

cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

In [ ]:
cat_list, _, _ = feature_type_split(data)
for col in cat_list:
    data[col] = data[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

data.replace('', np.nan, inplace=True)
null_percentages = (data[cat_list].isnull().sum() / len(data)) * 100

print("Null percentages:")
print(null_percentages)

In [ ]:
msno.bar(data[cat_list])

In [ ]:
plt.figure(figsize=(18, 8))
colours = ['#34495E', 'seagreen']
sns.heatmap(data[cat_list].isnull(), cmap=sns.color_palette(colours))
plt.title('Heatmap of Missing Values for Object Columns')
plt.show()

1. NAME_TYPE_SUITE_app (MCAR) : no pattern / correlation with its own column or other columns (heatmaps and bar charts) => Complete case analysis
2. OCCUPATION_TYPE_app (MCAR) : no pattern / correlation with its own column or other columns (heatmaps and bar charts) => Complete case analysis => considerable amount of rows => Column detetion
3. FONDKAPREMONT_MODE_app, HOUSETYPE_MODE_app, WALLSMATERIAL_MODE_app, EMERGENCYSTATE_MODE_app (MAR) : pattern / correlation between the columns => Single imputation methods not valid => Multiple imputation needed => less directly applicable to categorical data than on numerical data => Column deletion
4. SK_ID_BUREAU_bur (MAR) : due to initial aggregation performed in order to do the grouping => Single imputation value ("missing") as we only use the variable for feature engineering purposes

In [ ]:
data = data.dropna(subset=['NAME_TYPE_SUITE_app'])
columns_to_remove = ['FONDKAPREMONT_MODE_app', 'HOUSETYPE_MODE_app', 'WALLSMATERIAL_MODE_app', 'EMERGENCYSTATE_MODE_app','OCCUPATION_TYPE_app']
data = data.drop(columns=columns_to_remove)
data['SK_ID_BUREAU_bur'] = data['SK_ID_BUREAU_bur'].fillna('missing')
print("Shape " + str(data.shape))

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

In [ ]:
plt.figure(figsize=(18, 8))
colours = [ 'seagreen','#34495E'] 
sns.heatmap(data[cat_list].isnull(), cmap=sns.color_palette(colours))
plt.title('Heatmap of Missing Values for Object Columns')
plt.show()

#### 2.4 Quantitative data - NonGraphical / Graphical - Data Cleaning

In [ ]:
data["DAYS_BIRTH_app"] = abs(data["DAYS_BIRTH_app"])
data["DAYS_BIRTH_app"] = data["DAYS_BIRTH_app"] / 365

fig = px.histogram(data, x='DAYS_BIRTH_app', title='Histogram of DAYS_BIRTH', width=800)
median_value = data['DAYS_BIRTH_app'].median()
mean_value = data['DAYS_BIRTH_app'].mean()
mode_value = data['DAYS_BIRTH_app'].mode()[0]

fig.add_vline(x=median_value, line_dash="dash", line_color="green", annotation_text="Median",annotation_font=dict(size=4))
fig.add_vline(x=mean_value, line_dash="dash", line_color="blue", annotation_text="Mean", annotation_font=dict(size=4))
fig.add_vline(x=mode_value, line_dash="dash", line_color="red", annotation_text="Mode", annotation_font=dict(size=4))

fig.show() 

In [ ]:
for column in data.select_dtypes(include=['number']).columns:
    data[column] = data[column].apply(lambda x: abs(x) if x != 0 else 0)

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data)
for col in dis_num_list + num_list:
    if data[col].dtype == 'object':
        data[col] = data[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

data.replace('', np.nan, inplace=True)

null_percentages_dis_num = (data[dis_num_list].isnull().sum() / len(data)) * 100
print("Null percentages for discrete numerical features:")
print(null_percentages_dis_num)
print('-----------------------------------------')

null_percentages_num = (data[num_list].isnull().sum() / len(data)) * 100
print("Null percentages for continuous numerical features:")
print(null_percentages_num)

In [ ]:
columns_to_convert = [ 
    'SK_ID_CURR', 'SK_ID_BUREAU_bur', 'FLAG_MOBIL_app', 
    'FLAG_EMP_PHONE_app', 'FLAG_WORK_PHONE_app', 'FLAG_CONT_MOBILE_app',
    'FLAG_PHONE_app', 'FLAG_EMAIL_app', 'REGION_RATING_CLIENT_app', 'REGION_RATING_CLIENT_W_CITY_app', 'REG_REGION_NOT_LIVE_REGION_app','REG_REGION_NOT_WORK_REGION_app',
    'LIVE_REGION_NOT_WORK_REGION_app', 'REG_CITY_NOT_LIVE_CITY_app', 'REG_CITY_NOT_WORK_CITY_app', 'LIVE_CITY_NOT_WORK_CITY_app'
]
for col in columns_to_convert: 
    data[col] = data[col].astype('object')

cat_list, dis_num_list, num_list = feature_type_split(data)

combined_list = dis_num_list + num_list
msno.bar(data[combined_list])

In [ ]:
numerical_data = data[combined_list]
plt.figure(figsize=(25, 8))
colours = ['#34495E', 'seagreen']
sns.heatmap(numerical_data.isnull(), cmap=sns.color_palette(colours))
plt.title('Heatmap of Missing Values for Numerical Columns')
plt.show()

1. DAYS_ENDDATE_FACT_bur, AMT_CREDIT_SUM_DEBT_bur, AMT_CREDIT_SUM_LIMIT_bur, DAYS_CREDIT_ENDDATE_bur (MCAR) => no pattern / correlation with its own column or other columns (heatmaps and bar charts) => Complete case analysis
2. OWN_CAR_AGE_app, AMT_CREDIT_MAX_OVERDUE_bur, AMT_ANNUITY_y (MCAR) => no pattern / correlation with its own column or other columns (heatmaps and bar charts) => Complete case analysis => considerable amount of rows => Column deletion
3. Other columns (MAR) => pattern / correlation between the columns => Single imputation methods not valid => Multiple imputation needed

In [ ]:
data = data.dropna(subset=['DAYS_ENDDATE_FACT_bur','AMT_CREDIT_SUM_DEBT_bur','AMT_CREDIT_SUM_LIMIT_bur','DAYS_CREDIT_ENDDATE_bur'])
columns_to_remove = ['OWN_CAR_AGE_app','AMT_CREDIT_MAX_OVERDUE_bur','AMT_ANNUITY_y']
data = data.drop(columns=columns_to_remove)
columns_with_missing_values = data.columns[data.isnull().any()].tolist()
imputer = IterativeImputer()
data[columns_with_missing_values] = imputer.fit_transform(data[columns_with_missing_values])
print("Shape " + str(data.shape))

In [ ]:
normal_cols = []

for column in data.select_dtypes(include=['float64', 'int64']).columns:
       
    stat, p_value = shapiro(data[column])  # Perform Shapiro-Wilk test
    if p_value >= 0.05:
        normal_cols.append(column)

print("Columns with normal distribution:")
print(normal_cols)

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

In [ ]:
combined_list = dis_num_list + num_list
numerical_data = data[combined_list]
plt.figure(figsize=(25, 8))
colours = ['seagreen','#34495E' ]
sns.heatmap(numerical_data.isnull(), cmap=sns.color_palette(colours))
plt.title('Heatmap of Missing Values for Numerical Columns')
plt.show()

In [ ]:
def identify_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = column[(column < lower_bound) | (column > upper_bound)]
    return outliers

print("Shape before removing outliers:", data.shape)

for column in num_list:
    outliers = identify_outliers_iqr(data[column])
    data = data[~data[column].isin(outliers)]

print("Shape after removing outliers 1:", data.shape)

for column in dis_num_list:
    outliers = identify_outliers_iqr(data[column])
    data = data[~data[column].isin(outliers)]

print("Shape after removing outliers 2:", data.shape)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
for column in num_list:
    data[[column]] = scaler.fit_transform(data[[column]])

### Analysis 3 - Bivariate Analysis - NonGraphical / Graphical

Due to the imbalance nature of the target variable, we assume that we cannot visually observe any clear patterns between "TARGET_app" and the other variables.

### Analysis 4 - Feature Selection

### 4.1 Correlation-based feature selection - Quantitative data

In [ ]:
data['TARGET_app'].value_counts()

In [ ]:
data['TARGET_app'] = data['TARGET_app'].replace({'non-default': 0, 'default': 1})
numeric_columns_df = data.select_dtypes(include=['number'])
numerical_corr_spearman = numeric_columns_df.corr(method='spearman')
numerical_corr_spearman = numerical_corr_spearman.style.map(lambda x: 'background-color: green' if abs(float(x)) > 0.80 else '') # Spearman is used because we assume that most numerical columns do not follow a normal distribution
numerical_corr_spearman

In [ ]:
numeric_columns_df = data.select_dtypes(include=['number'])
numerical_corr_spearman = numeric_columns_df.corr(method='spearman')
sorted_corr = numerical_corr_spearman['TARGET_app'].abs().sort_values(ascending=False)
top_10_features = sorted_corr.head(11)[1:]  # Excluding 'TARGET'

print("Top 10 Numerical Features Correlated with 'TARGET':")
print(top_10_features)

In [ ]:
numeric_columns_df = data.select_dtypes(include=['number'])
numerical_corr_spearman = numeric_columns_df.corr(method='spearman')
high_corr_pairs = (numerical_corr_spearman.abs() > 0.80) & (numerical_corr_spearman.abs() < 1)

columns_to_delete = set()

for col1 in high_corr_pairs.columns:
    for col2 in high_corr_pairs.index:
        if high_corr_pairs.loc[col2, col1]:
            if abs(numerical_corr_spearman.loc[col1, 'TARGET_app']) > abs(numerical_corr_spearman.loc[col2, 'TARGET_app']):
                columns_to_delete.add(col2)
            else:
                columns_to_delete.add(col1)

columns_removed = list(columns_to_delete)
data.drop(columns=columns_to_delete, inplace=True)

print("Columns Removed:")
for col in columns_removed:
    print(col)

print("\nUpdated DataFrame:")
data.head()

### 4.2 Removing features with low variance

In [ ]:
numerical_cols = data.select_dtypes(exclude=['object']).columns
selector = VarianceThreshold(threshold=0) # constant
selected_features = selector.fit_transform(data[numerical_cols])
support = selector.get_support()
variances = selector.variances_
cols_to_remove = numerical_cols[~support]
data = data.drop(columns=cols_to_remove)
print("Numerical Columns to Remove based on Variance Threshold:")
print(cols_to_remove)

### 4.3 Correlation-based feature selection - Qualitative data

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

In [ ]:
df_categorical = data[cat_list]
corr_measures = associations(df_categorical, nominal_columns='all', plot=False)
fig = px.imshow(corr_measures['corr'], color_continuous_scale="RdBu", text_auto=True)
fig.update_layout(title="Cramer's V Correlation Heatmap", width=800, height=700)
fig.show()

In [ ]:
del data['LIVE_REGION_NOT_WORK_REGION_app']
del data['REG_CITY_NOT_WORK_CITY_app']
del data['REGION_RATING_CLIENT_W_CITY_app']

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

In [ ]:
del data['SK_ID_BUREAU_bur']
data.set_index('SK_ID_CURR', inplace=True)
data.head()

In [ ]:
columns_list = list(data.columns)
print(columns_list)

### Analysis 5 - Feature Encoding

In [ ]:
le = LabelEncoder()
le_count = 0
le_encoded_cols = []  

for col in data.columns[1:]:
    if data[col].dtype == 'object':
        if len(list(data[col].unique())) <= 2:
            le.fit(data[col])
            data[col] = le.transform(data[col])
            le_count += 1
            le_encoded_cols.append(col) 

print('{} columns were label encoded.'.format(le_count))
print('Columns label encoded: {}'.format(', '.join(le_encoded_cols))) 

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

In [ ]:
features_to_one_hot_encode = [feature for feature in cat_list if data[feature].nunique() > 2]

one_hot_df = pd.get_dummies(data[features_to_one_hot_encode])
one_hot_df = one_hot_df.astype(int)

data = pd.concat([data.drop(columns=[*features_to_one_hot_encode]), one_hot_df], axis=1)

In [ ]:
data['TARGET_app'] = le.fit_transform(data['TARGET_app'])

In [ ]:
cat_list, dis_num_list, num_list = feature_type_split(data) 
print(str(len(cat_list)),'categorical features:', cat_list)
print('-----------------------------------------')
print(str(len(dis_num_list)),'discrete numerical features:',dis_num_list)
print('-----------------------------------------')
print(str(len(num_list)),'continuous numerical features:',num_list)

### Analysis 6 - Modelling

In [ ]:
y = data['TARGET_app']
X = data.drop(columns=['TARGET_app'])
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, stratify=y, random_state=8) # X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state = 8)
X_test, X_calibration, y_test, y_calibration = train_test_split(X_remaining, y_remaining, test_size=0.5, stratify=y_remaining, random_state=8)

In [ ]:
# X_remaining.to_csv('X_remaining_cp.csv', index=True)
y_remaining_df = pd.DataFrame({'TARGET_app': y_remaining})
# y_remaining_df.to_csv('y_remaining_cp.csv', index=True)

# X_test.to_csv('X_test_cp.csv', index=True)
y_test_df = pd.DataFrame({'TARGET_app': y_test})
# y_test_df.to_csv('y_test_cp.csv', index=True)
# X_calibration.to_csv('X_calibration_cp.csv', index=True)
y_calibration_df = pd.DataFrame({'TARGET_app': y_calibration})
# y_calibration_df.to_csv('y_calibration_cp.csv', index=True)

In [ ]:
smote = SMOTE(random_state=8)
X_train, y_train= smote.fit_resample(X_train, y_train)

print(f"Resampled training features shape: {X_train.shape}")
print(f"Resampled training target shape: {y_train.shape}")

In [ ]:
cv = KFold(n_splits=10)

def calculate_mcc(y_true, y_pred):
    return matthews_corrcoef(y_true, y_pred) if len(np.unique(y_true)) > 1 else 0.0

precision_scores = []
recall_scores = []
mcc_scores = []

for train, val in cv.split(X_train, y_train):

    X_train_fold, X_val_fold = X_train.iloc[train], X_train.iloc[val]
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    model_x = RandomForestClassifier(random_state=42)
    model_x.fit(X_train_fold, y_train_fold)
    y_pred_val_fold = model_x.predict(X_val_fold)
    
    precision_fold = precision_score(y_val_fold, y_pred_val_fold)
    precision_scores.append(precision_fold)
    
    recall_fold = recall_score(y_val_fold, y_pred_val_fold)
    recall_scores.append(recall_fold)
    
    mcc_fold = calculate_mcc(y_val_fold, y_pred_val_fold)
    mcc_scores.append(mcc_fold)

average_precision_score = np.mean(precision_scores)
average_recall_score = np.mean(recall_scores)
average_mcc_score = np.mean(mcc_scores)

print("Average precision: {:.4f}".format(average_precision_score))
print("Average recall: {:.4f}".format(average_recall_score))
print("Average MCC: {:.4f}".format(average_mcc_score))

In [ ]:
cv = KFold(n_splits=10)

precision_scores = []
recall_scores = []
mcc_scores = []

for train, val in cv.split(X_train, y_train):

    X_train_fold, X_val_fold = X_train.iloc[train], X_train.iloc[val]
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    model_x = XGBClassifier(random_state=42)
    model_x.fit(X_train_fold, y_train_fold)
    y_pred_val_fold = model_x.predict(X_val_fold)
    
    precision_fold = precision_score(y_val_fold, y_pred_val_fold)
    precision_scores.append(precision_fold)
    
    recall_fold = recall_score(y_val_fold, y_pred_val_fold)
    recall_scores.append(recall_fold)
    
    mcc_fold = calculate_mcc(y_val_fold, y_pred_val_fold)
    mcc_scores.append(mcc_fold)

average_precision_score = np.mean(precision_scores)
average_recall_score = np.mean(recall_scores)
average_mcc_score = np.mean(mcc_scores)

print("Average precision: {:.4f}".format(average_precision_score))
print("Average recall: {:.4f}".format(average_recall_score))
print("Average MCC: {:.4f}".format(average_mcc_score))

In [ ]:
X_train.columns = ["".join(c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]

cv = KFold(n_splits=10)

precision_scores = []
recall_scores = []
mcc_scores = []

for train, val in cv.split(X_train, y_train):

    X_train_fold, X_val_fold = X_train.iloc[train], X_train.iloc[val]
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    model_lgb = LGBMClassifier(random_state=42, verbose = 0)
    model_lgb.fit(X_train_fold, y_train_fold)
    y_pred_val_fold = model_lgb.predict(X_val_fold)
    
    precision_fold = precision_score(y_val_fold, y_pred_val_fold)
    precision_scores.append(precision_fold)
    
    recall_fold = recall_score(y_val_fold, y_pred_val_fold)
    recall_scores.append(recall_fold)
    
    mcc_fold = calculate_mcc(y_val_fold, y_pred_val_fold)
    mcc_scores.append(mcc_fold)

average_precision_score = np.mean(precision_scores)
average_recall_score = np.mean(recall_scores)
average_mcc_score = np.mean(mcc_scores)

print("Average precision: {:.4f}".format(average_precision_score))
print("Average recall: {:.4f}".format(average_recall_score))
print("Average MCC: {:.4f}".format(average_mcc_score))

In [ ]:
scorer = make_scorer(matthews_corrcoef)

param_grid = {'randomforestclassifier__n_estimators': [50, 100, 200],
              'randomforestclassifier__max_depth': [10, 20, 30, 40, 50],
              'randomforestclassifier__min_samples_split': [2, 5, 10],
              'randomforestclassifier__min_samples_leaf': [1, 2, 4]} 

k = 10

cv = KFold(n_splits=k)
rf = RandomForestClassifier(random_state=42)

pipeline = Pipeline([('randomforestclassifier', rf)])
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring=scorer)
grid_search.fit(X_train, y_train)

print("Best hyperparameters: ", grid_search.best_params_)
print("Best MCC score: ", grid_search.best_score_)

# Best hyperparameters:  {'randomforestclassifier__max_depth': 50, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__n_estimators': 200}
# Best MCC score:  0.33353458327828533

In [ ]:
scorer = make_scorer(matthews_corrcoef)

param_grid = {'xgbclassifier__n_estimators': [50, 100, 200],
              'xgbclassifier__max_depth': [3, 4, 5, 6],
              'xgbclassifier__learning_rate': [0.01, 0.05, 0.1]} 

k = 10
cv = KFold(n_splits=k)

xgb = XGBClassifier(random_state=42)

pipeline = Pipeline([('xgbclassifier', xgb)])
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring=scorer)
grid_search.fit(X_train, y_train)

print("Best hyperparameters: ", grid_search.best_params_)
print("Best MCC score: ", grid_search.best_score_)

# Best hyperparameters:  {'xgbclassifier__learning_rate': 0.05, 'xgbclassifier__max_depth': 6, 'xgbclassifier__n_estimators': 200}
# Best MCC score:  0.15417460213744283

In [ ]:
logging.getLogger('lightgbm').setLevel(logging.ERROR)
scorer = make_scorer(matthews_corrcoef)

param_grid = {'lgbmclassifier__n_estimators': [50, 100, 200],
              'lgbmclassifier__max_depth': [3, 4, 5, 6],
              'lgbmclassifier__learning_rate': [0.01, 0.05, 0.1]} 

k = 10
cv = KFold(n_splits=k)

lgbm = LGBMClassifier(random_state=42, verbose=0)
pipeline = Pipeline([('lgbmclassifier', lgbm)])

warnings.filterwarnings("ignore")

grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring=scorer)
grid_search.fit(X_train, y_train)

print("Best hyperparameters: ", grid_search.best_params_)
print("Best MCC score: ", grid_search.best_score_)

#Best hyperparameters:  {'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': 5, 'lgbmclassifier__n_estimators': 100}
#Best MCC score:  0.14678247564181304

In [ ]:
params = {
    'max_depth': 50,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'n_estimators': 200,
    'random_state': 42
}

def calculate_mcc(y_true, y_pred):
    return matthews_corrcoef(y_true, y_pred) if len(np.unique(y_true)) > 1 else 0.0

cv = KFold(n_splits=10)

precision_scores = []
recall_scores = []
mcc_scores = []

for train, val in cv.split(X_train, y_train):

    X_train_fold, X_val_fold = X_train.iloc[train], X_train.iloc[val]
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    model_rf = RandomForestClassifier(**params)
    model_rf.fit(X_train_fold, y_train_fold)
    y_pred_val_fold = model_rf.predict(X_val_fold)
    
    precision_fold = precision_score(y_val_fold, y_pred_val_fold)
    precision_scores.append(precision_fold)
    
    recall_fold = recall_score(y_val_fold, y_pred_val_fold)
    recall_scores.append(recall_fold)
    
    mcc_fold = calculate_mcc(y_val_fold, y_pred_val_fold)
    mcc_scores.append(mcc_fold)

average_precision_score = np.mean(precision_scores)
average_recall_score = np.mean(recall_scores)
average_mcc_score = np.mean(mcc_scores)

print("Average precision: {:.4f}".format(average_precision_score))
print("Average recall: {:.4f}".format(average_recall_score))
print("Average MCC: {:.4f}".format(average_mcc_score))

In [ ]:
model_rf.fit(X_train, y_train)
# joblib.dump(model_rf, 'model_rf.joblib')

In [ ]:
y_pred = model_rf.predict(X_test)
y_prob = model_rf.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_prob)
print(f"AUC Score: {auc_score:.4f}")

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])

fig = px.imshow(cm_df, labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['Predicted Negative', 'Predicted Positive'],
                y=['Actual Negative', 'Actual Positive'],
                color_continuous_scale=px.colors.sequential.Blues,text_auto=True)

fig.update_traces(showscale=True) 
fig.update_layout(title='Confusion Matrix')
fig.show()

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=1, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

### Analysis 7 - Fairness Analysis

In [ ]:
A = X_test["CODE_GENDER_app"]
A_str = A.map({ 1:"male", 0:"female"})

metrics_dict = {
        "Demographic parity difference": demographic_parity_difference(y_test, y_pred, sensitive_features=A_str),
        "Equalized odds difference": equalized_odds_difference(y_test, y_pred, sensitive_features=A_str),
    }

pd.DataFrame.from_dict(metrics_dict, orient="index", columns=["Metrics"])